## Real Time Testing

In [1]:
# ! pip install cvzone==1.5.6
# ! pip install opencv-python==4.7.0.72
# ! pip install numpy==1.26.2
# ! pip install mediapipe==0.9.1.0
# ! pip install tensorflow==2.11.0
# ! pip install wordninja==2.0.0
# ! pip install transformer==4.35.2
# ! pip install kivy==2.1.0
# ! pip install kivymd==1.1.0

In [2]:
from collections import deque
from concurrent.futures import ThreadPoolExecutor
from time import time
from cvzone import FPS

import cv2
import numpy as np
from itertools import chain
from time import time

from cvzone.HandTrackingModule import HandDetector
from cvzone.FaceDetectionModule import FaceDetector
from cvzone.PoseModule import PoseDetector

In [3]:
## 2. Feature Extraction (Hand+Face+Pose Detection)
# Flatten a 2d np array into 1d array
def flatten2dList(arr, dataType=int):
    return np.fromiter(chain.from_iterable(arr), dataType)

# Get the largest absolute value in an np array
def getAbsLargestVal(arr):
    return np.max(np.abs(arr))

# Offset and normalize the landmark list
# Returns a 1d numpy array
def preprocess_landmarks(landmark_list):    
    landmark_list = np.array(landmark_list, dtype=float)
    origin = landmark_list[0]
    
    # Offset every point with respect to the first point
    # Convert to 1D-array
    new_landmark_list = (landmark_list - origin).ravel()
    
    # Get highest absolute value
    largest_value = getAbsLargestVal(new_landmark_list)
    
    # Normalization
    if largest_value != 0:
        return new_landmark_list / largest_value
    return new_landmark_list

# Offset and normalize a BBOX list (BBOX = Bounding Box, used in face and hand detection)
# Returns a 1d numpy array
def preprocess_bbox(bbox, frameSize):
    bbox = np.array(bbox, dtype=float)
    # Convert 3rd and 4th element into coordinates instead of width/height
    bbox[2] = bbox[0] + bbox[2]
    bbox[3] = bbox[1] + bbox[3]

    # Normalize against frame size
    bbox[0] /= frameSize[0]
    bbox[1] /= frameSize[1]
    bbox[2] /= frameSize[0]
    bbox[3] /= frameSize[1]

    return bbox

# Normalize a center vertex (a list of 2 elements)
# Returns a 1d numpy array
def preprocess_center(center, frameSize):
    center = np.array(center, dtype=float)
    center[0] /= frameSize[0]
    center[1] /= frameSize[1]
    return center

# Preprocess (Offset and normalize) the body's landmark list, bbox and center
def preprocess_body_part(bodyPart, frameSize):
    bodyPart['lmList'] = preprocess_landmarks(bodyPart['lmList'])
    bodyPart['bbox'] = preprocess_bbox(bodyPart['bbox'], frameSize)
    bodyPart['center'] = preprocess_center(bodyPart['center'], frameSize)
    return bodyPart

# Function to generate empty/placeholder data for a hand 
# Used when a hand is not detected in frame
def generate_empty_hand(type):
    return {
        'lmList': np.zeros(21 * 3, dtype=int), 
        'bbox': np.zeros(4, dtype=float), 
        'center': np.zeros(2, dtype=float), 
        'type': type
    }

# Select the best matching face, aka the one with the best score (clarity)
# and closest to the center of the screen
# Since the Neural network will be design to only accept one face
def select_best_matching_face(faces, frameSize):
    if not faces or len(faces) == 0:
        return False
    elif len(faces) == 1:
        return faces[0]
    
    def difference(a, b):
        return ((a[0] - b[0])**2) + ((a[1] - b[1])**2)
    
    frameCenter = (frameSize[0] / 2, frameSize[1] / 2)

    best_score = faces[0]
    best_center = faces[0]
    center_diff = difference(faces[0]['center'], frameCenter)

    for each in faces:
        if difference(each['center'], frameCenter) < center_diff:
            best_center = each
        if each['score'][0] > best_score['score'][0]:
            best_score = each
    
    if best_center['score'][0] > 0.5:
        return best_center
    return best_score

# Flatten everything
def flattenDetectionResult(obj):
    # return np.fromiter(chain.from_iterable([obj['lmList'], obj['bbox'], obj['center']]), float)
    return np.concatenate([obj['lmList'], obj['bbox'], obj['center']])

In [4]:
# Detects hands, face & pose, 
# convert them into normalized landmark/keypoint coordinates in a 1D-array, 
# and also returns the frame with the landmark connections drawn onto it

# Improved/Parallelised version
def featureExtractionV3(handDetector, faceDetector, poseDetector, frame, draw=True):
    def detectHands(handDetector, frame, frameSize, draw):
        results = None
        # Hand Detection
        if (draw):
            results, frame = handDetector.findHands(frame, draw=draw)
        else:
            results = handDetector.findHands(frame, draw=draw)

        if not results:
            results = [generate_empty_hand('Left'), generate_empty_hand('Right')]
        elif len(results) == 1:
            if (results[0]['type'] == 'Left'):
                results[0] = preprocess_body_part(results[0], frameSize)
                results.append(generate_empty_hand('Right'))
            else:
                results[0] = preprocess_body_part(results[0], frameSize)
                results.insert(0, generate_empty_hand('Left'))                         
        else:
            results[0] = preprocess_body_part(results[0], frameSize)
            results[1] = preprocess_body_part(results[1], frameSize)
        return results

    # Pose Detection
    # **We only use the first 23 out of the total 33 landmark points 
    #   as those represent the lower half body and are irrelevant to sign language interpretation
    def detectPose(poseDetector, frame, draw):
        frame = poseDetector.findPose(frame, draw=draw)
        results, _ = poseDetector.findPosition(frame, bboxWithHands=False)
        if results:
            results = preprocess_landmarks(results[:23])
        else:
            results = np.zeros(23, dtype=int)
        return results
    
    # Face Detection
    def detectFace(faceDetector, frame, frameSize, draw):
        frame, results = faceDetector.findFaces(frame, draw=draw)
        if results:
            results = select_best_matching_face(results, frameSize)
            results['bbox'] = preprocess_bbox(results['bbox'], frameSize)
            results['center'] = preprocess_center(results['center'], frameSize)
        else:
            results = {
                'bbox': np.zeros(4, dtype=float), 
                'center': np.zeros(2, dtype=float)
            }
        return results

    frameSize = (frame.shape[1], frame.shape[0])
    with ThreadPoolExecutor() as executor:
        t1 = executor.submit(detectHands, handDetector, frame, frameSize, draw)
        t2 = executor.submit(detectPose, poseDetector, frame, draw)
        t3 = executor.submit(detectFace, faceDetector, frame, frameSize, draw)
        
        # Convert results into 1D-array
        detectionResults = flatten2dList([
            flattenDetectionResult(t1.result()[0]), 
            flattenDetectionResult(t1.result()[1]), 
            t2.result(), 
            t3.result()['bbox'],
            t3.result()['center'],
            t3.result()['center'] - t1.result()[0]['center'],
            t3.result()['center'] - t1.result()[1]['center']
        ], dataType=float)

        return detectionResults, frame

In [5]:
# Detects hands, face & pose, 
# convert them into normalized landmark/keypoint coordinates in a 1D-array, 
# and also returns the frame with the landmark connections drawn onto it

# Improved/Parallelised version
def staticFeatureExtraction(handDetector, frame, draw=True):
    def detectHands(handDetector, frame, frameSize, draw):
        results = None
        # Hand Detection
        if (draw):
            results, frame = handDetector.findHands(frame, draw=draw)
        else:
            results = handDetector.findHands(frame, draw=draw)

        if not results:
            results = [generate_empty_hand('Left'), generate_empty_hand('Right')]
        elif len(results) == 1:
            if (results[0]['type'] == 'Left'):
                results[0] = preprocess_body_part(results[0], frameSize)
                results.append(generate_empty_hand('Right'))
            else:
                results[0] = preprocess_body_part(results[0], frameSize)
                results.insert(0, generate_empty_hand('Left'))                         
        else:
            results[0] = preprocess_body_part(results[0], frameSize)
            results[1] = preprocess_body_part(results[1], frameSize)
        return results

    frameSize = (frame.shape[1], frame.shape[0])
    with ThreadPoolExecutor() as executor:
        t1 = executor.submit(detectHands, handDetector, frame, frameSize, draw)
        # t2 = executor.submit(detectPose, poseDetector, frame, draw)
        # t3 = executor.submit(detectFace, faceDetector, frame, frameSize, draw)
        
        # Convert results into 1D-array
        detectionResults = flatten2dList([
            flattenDetectionResult(t1.result()[0]), 
            flattenDetectionResult(t1.result()[1]), 
            # t2.result(), 
            # t3.result()['bbox'],
            # t3.result()['center'],
            # t3.result()['center'] - t1.result()[0]['center'],
            # t3.result()['center'] - t1.result()[1]['center']
        ], dataType=float)

        return detectionResults, frame

In [8]:
import keras
from files_io import readActionLabels

# Load the model from the H5 file
model = keras.models.load_model('../static_recognition/models/static_model_V1.h5')
# model = keras.models.load_model('../static_recognition/models/static_model.keras')

# Print the summary before loading
model.summary()

static_labels = readActionLabels()
static_labels

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 136, 32)           128       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 68, 32)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 66, 64)            6208      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 33, 64)           0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 31, 128)           24704     
                                                                 
 flatten (Flatten)           (None, 3968)              0

['A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10']

In [12]:
cam = cv2.VideoCapture(0, cv2.CAP_DSHOW)
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

# Detectors
handDetector = HandDetector(detectionCon=0.5, maxHands=2)
faceDetector = FaceDetector(minDetectionCon=0.5)
poseDetector = PoseDetector(detectionCon=0.5)
fpsReader = FPS()

timeStats = []

try:
    predictionHistory = deque()
    detectionThreshold = 0.999    
    predictionCooldown = 0.5
    appendCooldown = 1.0
    # the first time append should eliminate predictionCooldown
    lastAppendTime = time() + appendCooldown
    lastDetectTime = time() + predictionCooldown

    while True:
        startTime = time()

        # Read from camera
        success, frame = cam.read()
        if not success:
            raise Exception("No Frames Read")
        frame = cv2.flip(frame, 1)

        # Hand Detection
        inputValue, frame = staticFeatureExtraction(
            handDetector, frame)
        
        # detectionResults, frame = featureExtractionV3(
        #     handDetector, faceDetector, poseDetector, frame)
        
        detectionResults = np.expand_dims(
            inputValue, axis=0) # reshape detection result
        
        if time() <= lastDetectTime + predictionCooldown:
            pass
        else:
            predictionResults = model.predict(
                x=detectionResults,
                verbose=0,
                use_multiprocessing=True,
                workers=4
            )[0]
            lastDetectTime = time()
            
            print(predictionResults)
            
            # Get predicted character and accuracy
            predCharacter = static_labels[np.argmax(predictionResults)]
            predAccuracy = predictionResults[np.argmax(predictionResults)]
        
            cv2.putText(frame, (predCharacter + ":"+ str(predAccuracy)), (15, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)          
            
            if predAccuracy >= detectionThreshold:            
                # If predictionHistory is not empty
                # If predCharacter is the same as the last appended character
                # Check if 0.5 seconds have passed since the last append
                if predictionHistory and predCharacter == predictionHistory[-1] and time() <= lastAppendTime + appendCooldown:
                    # Do nothing, don't append
                    pass
                else:
                    predictionHistory.append(predCharacter)
                    # Reset the timestamp when a new character is detected
                    lastAppendTime = time()

                if len(predictionHistory) > 5:
                        predictionHistory.popleft()  
                        
        # cv2.putText(frame, (predCharacter + ":"+ str(predAccuracy)), (15, 70),
        #             cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)
                        
        cv2.putText(frame, str(predictionHistory), (15, 120),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)
        

        fps, frame = fpsReader.update(frame, pos=(
            950, 80), color=(0, 255, 0),scale=5, thickness=5)

        # fps, frame = fpsReader.update(frame, pos=(
        #     950, 80), color=(0, 255, 0), scale=5, thickness=5)

        
        # Show resulting frame
        # cv2.putText(frame, f'Training #{training + 1} for \'{action}\'', (15, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
        cv2.imshow("Sign Language Recognition Prototype", frame)

        timeStats.append(time() - startTime)

        keyPressed = cv2.waitKey(1)
        # Stop Program when pressed 'Esc'
        if keyPressed == 27:
            raise Exception("Finished")
        elif keyPressed == ord('c'):
            predictionHistory.clear()
            lastAppendTime = time() + predictionCooldown


except Exception as e:
    cam.release()
    cv2.destroyAllWindows()
    print(e)

cam.release()
cv2.destroyAllWindows()

[1.8090052e-05 2.4316146e-03 1.3968395e-07 9.6068042e-04 2.9915303e-04
 2.0861217e-01 1.8979903e-05 8.6546288e-04 2.6629789e-02 2.0735088e-04
 4.2041745e-03 1.5665263e-01 3.1518698e-04 4.0926156e-05 5.3178850e-05
 1.1235599e-02 2.7501036e-03 3.9008239e-05 3.3178353e-06 2.5521650e-04
 9.5262043e-03 7.4681950e-05 1.7810527e-05 1.7497676e-03 2.5561586e-04
 3.6476322e-03 7.8179970e-02 1.4439473e-04 1.3553315e-04 1.9162690e-04
 2.8740210e-03 1.1941707e-03 2.1455996e-04 4.2086517e-04 1.2970447e-04
 4.8565066e-01]
[1.8090052e-05 2.4316146e-03 1.3968395e-07 9.6068042e-04 2.9915303e-04
 2.0861217e-01 1.8979903e-05 8.6546288e-04 2.6629789e-02 2.0735088e-04
 4.2041745e-03 1.5665263e-01 3.1518698e-04 4.0926156e-05 5.3178850e-05
 1.1235599e-02 2.7501036e-03 3.9008239e-05 3.3178353e-06 2.5521650e-04
 9.5262043e-03 7.4681950e-05 1.7810527e-05 1.7497676e-03 2.5561586e-04
 3.6476322e-03 7.8179970e-02 1.4439473e-04 1.3553315e-04 1.9162690e-04
 2.8740210e-03 1.1941707e-03 2.1455996e-04 4.2086517e-04 1.29

In [ ]:
timeStats[10:]

[0.0817103385925293,
 0.09274458885192871,
 0.09378242492675781,
 0.13100481033325195,
 0.20148301124572754,
 0.0344235897064209,
 0.04212641716003418,
 0.09451961517333984,
 0.09219956398010254,
 0.09090900421142578,
 0.2041645050048828,
 0.03135490417480469,
 0.06314301490783691,
 0.12901711463928223,
 0.07598423957824707,
 0.09213805198669434,
 0.18729734420776367,
 0.0715949535369873,
 0.061200857162475586,
 0.09507298469543457,
 0.11060357093811035,
 0.09206223487854004,
 0.15856361389160156,
 0.05353498458862305,
 0.0865485668182373,
 0.09316635131835938,
 0.09557580947875977,
 0.10957884788513184,
 0.1618030071258545,
 0.051770687103271484,
 0.08719873428344727,
 0.09527134895324707,
 0.09255361557006836,
 0.09357047080993652,
 0.21526551246643066,
 0.07152485847473145,
 0.031998634338378906,
 0.0725555419921875,
 0.09264779090881348,
 0.09569787979125977,
 0.11291813850402832,
 0.15155720710754395,
 0.0339963436126709,
 0.09053468704223633,
 0.10688424110412598,
 0.090357303619